In [2]:
import os 
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from scipy.io import loadmat

# Import pyod packages and methods

In [6]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

# import Metrics Package

In [8]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

# Define data file and read X and Y

In [10]:
mat_fie_list = ['arrhythmia.mat', 'cardio.mat', 'glass.mat', 'ionosphere.mat', 'letter.mat', 'lympho.mat', 'mnist.mat', 'musk.mat', 'optdigits.mat',
                'pendigits.mat', 'pima.mat', 'satellite.mat', 'satimage-2.mat',
                'shuttle.mat', 'vertebral.mat', 'vowels.mat','wbc.mat']

In [11]:
data = loadmat('cardio.mat')

In [12]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [13]:
len(data)

5

In [14]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [15]:
data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

# Input Feature Shape in Mat file format

In [16]:
type(data['X']),data['X'].shape

(numpy.ndarray, (1831, 21))

# Dependent / Target / Output Feature Shape

In [18]:
type(data['y']),data['y'].shape

(numpy.ndarray, (1831, 1))

# Define nine outlier detection tools to be compared

In [19]:
df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc', 'ABOD',
              'CBLOF', 'FB', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM', 'PCA']

In [20]:
roc_df = pd.DataFrame(columns=df_columns)

prn_df = pd.DataFrame(columns=df_columns)

time_df = pd.DataFrame(columns=df_columns)

# ROC Performance evaluation table

In [21]:
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,IForest,KNN,LOF,MCD,OCSVM,PCA


# precision_n_scorese - Performane evoluation table

In [22]:
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,IForest,KNN,LOF,MCD,OCSVM,PCA


# Time Dataframe

In [23]:
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,IForest,KNN,LOF,MCD,OCSVM,PCA


# Explorainf All Mat Files

In [32]:
from time import time
random_state = np.random.RandomState(42)
for mat_file in mat_fie_list:
    print("\n....Processing", mat_file, '...')
    mat = loadmat(os.path.join('data', mat_file))
    
    x = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    
    # construct container for saving result
    roc_list = [mat_file[:-4], x.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], x.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], x.shape[1], outliers_percentage]
    
    # 60% for traning and 40% for testing
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=random_state)
    
    # standardizing data for processing
    x_train_norm, x_test_norm = standardizer(x_train, x_test)
    
    classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(contamination=outliers_fraction),
                   'cluster-based Local Outlier Factor': CBLOF(
                    contamination=outliers_fraction, check_estimator=False,
                   random_state=random_state),
                  'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                                   random_state=random_state),
                  'Histogram-base Outlier Detection (HBOS)': HBOS(
                  contamination=outliers_fraction),
                  'Isolation Forest': IForest(contamination=outliers_fraction,
                                             random_state=random_state),
                  'k Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
                  'Local Outlier Factor (LOF)': LOF(contamination=outliers_fraction),
                  'Minimum Covariance Determinant (MCD)': MCD(
                      contamination=outliers_fraction, random_state=random_state),
                  'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
                  'Principal Component Analysis (PCA)': PCA(
                  contamination=outliers_fraction, random_state=random_state),
                  }
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(x_train_norm)
        test_scores = clf.decision_function(x_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)
        
        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)
        
        print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
              'execution time: (duration)s'.format(
              clf_name=clf_name, roc=roc,prn=prn, duration=duration))
        
        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


....Processing arrhythmia.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7687, precision @ rank n:0.3571, execution time: (duration)s
cluster-based Local Outlier Factor ROC:0.7684, precision @ rank n:0.4643, execution time: (duration)s
Feature Bagging ROC:0.7799, precision @ rank n:0.5, execution time: (duration)s
Histogram-base Outlier Detection (HBOS) ROC:0.8511, precision @ rank n:0.5714, execution time: (duration)s
Isolation Forest ROC:0.8527, precision @ rank n:0.5714, execution time: (duration)s
k Nearest Neighbors (KNN) ROC:0.782, precision @ rank n:0.5, execution time: (duration)s
Local Outlier Factor (LOF) ROC:0.7787, precision @ rank n:0.4643, execution time: (duration)s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.8228, precision @ rank n:0.4286, execution time: (duration)s
One-class SVM (OCSVM) ROC:0.7986, precision @ rank n:0.5, execution time: (duration)s
Principal Component Analysis (PCA) ROC:0.7997, precision @ rank n:0.5, execution time: (duration)s

....Processing cardio.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.5763, precision @ rank n:0.1875, execution time: (duration)s
cluster-based Local Outlier Factor ROC:0.8221, precision @ rank n:0.4844, execution time: (duration)s
Feature Bagging ROC:0.4879, precision @ rank n:0.1406, execution time: (duration)s
Histogram-base Outlier Detection (HBOS) ROC:0.8453, precision @ rank n:0.4688, execution time: (duration)s
Isolation Forest ROC:0.9414, precision @ rank n:0.5, execution time: (duration)s
k Nearest Neighbors (KNN) ROC:0.6959, precision @ rank n:0.2812, execution time: (duration)s
Local Outlier Factor (LOF) ROC:0.4715, precision @ rank n:0.125, execution time: (duration)s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-152.770956835748848 > -250.696836140088948). You may want to try with a higher value of support_fraction (current value: 0.511).
  RuntimeWarning)


Minimum Covariance Determinant (MCD) ROC:0.8778, precision @ rank n:0.3906, execution time: (duration)s
One-class SVM (OCSVM) ROC:0.9507, precision @ rank n:0.5938, execution time: (duration)s
Principal Component Analysis (PCA) ROC:0.9638, precision @ rank n:0.6875, execution time: (duration)s

....Processing glass.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7104, precision @ rank n:0.25, execution time: (duration)s
cluster-based Local Outlier Factor ROC:0.8506, precision @ rank n:0.25, execution time: (duration)s
Feature Bagging ROC:0.7043, precision @ rank n:0.25, execution time: (duration)s
Histogram-base Outlier Detection (HBOS) ROC:0.6524, precision @ rank n:0.0, execution time: (duration)s
Isolation Forest ROC:0.7195, precision @ rank n:0.25, execution time: (duration)s
k Nearest Neighbors (KNN) ROC:0.7805, precision @ rank n:0.25, execution time: (duration)s
Local Outlier Factor (LOF) ROC:0.7774, precision @ rank n:0.25, execution time: (duration)s
Minimum Covariance Deter

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.3486, precision @ rank n:0.0, execution time: (duration)s
One-class SVM (OCSVM) ROC:0.4972, precision @ rank n:0.0, execution time: (duration)s
Principal Component Analysis (PCA) ROC:0.504, precision @ rank n:0.0, execution time: (duration)s

....Processing pendigits.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7008, precision @ rank n:0.0308, execution time: (duration)s
cluster-based Local Outlier Factor ROC:0.9609, precision @ rank n:0.3077, execution time: (duration)s
Feature Bagging ROC:0.4687, precision @ rank n:0.0462, execution time: (duration)s
Histogram-base Outlier Detection (HBOS) ROC:0.9294, precision @ rank n:0.2615, execution time: (duration)s
Isolation Forest ROC:0.9422, precision @ rank n:0.2769, execution time: (duration)s
k Nearest Neighbors (KNN) ROC:0.7602, precision @ rank n:0.0462, execution time: (duration)s
Local Outlier Factor (LOF) ROC:0.481, precision @ rank n:0.0462, execution time: (duration)s
Minimum Covarian

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.541490230701029 > -75.592253998160075). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.022992269210960 > -76.150022952275933). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.588090860494539 > -73.997089087121466). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\P

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.288199308055468 > -75.990589261846011). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.329968951469141 > -77.419260242705974). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.221834205302052 > -78.090864446968652). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\P

Minimum Covariance Determinant (MCD) ROC:0.9903, precision @ rank n:0.7534, execution time: (duration)s
One-class SVM (OCSVM) ROC:0.9922, precision @ rank n:0.9553, execution time: (duration)s
Principal Component Analysis (PCA) ROC:0.9902, precision @ rank n:0.9503, execution time: (duration)s

....Processing vertebral.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.2797, precision @ rank n:0.0, execution time: (duration)s
cluster-based Local Outlier Factor ROC:0.3908, precision @ rank n:0.0, execution time: (duration)s
Feature Bagging ROC:0.3027, precision @ rank n:0.0, execution time: (duration)s
Histogram-base Outlier Detection (HBOS) ROC:0.2695, precision @ rank n:0.0, execution time: (duration)s
Isolation Forest ROC:0.3576, precision @ rank n:0.0, execution time: (duration)s
k Nearest Neighbors (KNN) ROC:0.318, precision @ rank n:0.0, execution time: (duration)s
Local Outlier Factor (LOF) ROC:0.318, precision @ rank n:0.0, execution time: (duration)s
Minimum Covariance Determina

In [34]:
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8309,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,32,6.25,0.8465,0.7423,0.866,0.5728,0.5778,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,mnist,100,9.2069,0.7813,0.8447,0.7259,0.5675,0.7801,0.8409,0.7085,0.863,0.8417,0.8396
0,musk,166,3.1679,0.0809,1,0.5228,0.9999,0.9996,0.7348,0.5323,1,1,1
0,arrhythmia,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8778,0.9507,0.9638
